In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#############################################
######### LOAD SUITE2P AND BINARIZE #########
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'
# data_dir = '/media/cat/4TB/donato/nathalie/plane0'
# data_dir = '/media/cat/4TB/donato/renan/renan'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228'   # can also add suite2p/plane0/
data_dir = '/media/cat/4TB/donato/renan/DON-012207/20230203/plane0/'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True                          # outputs additional information during processing
c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = data_dir
c.load_suite2p()                          

# set flags to save matlab and python data
c.save_python = True         # save output as .npz file 
c.save_matlab = False         # save output as .mat file

###############################################
##### PARAMETERS FOR RUNNING BINARIZATION #####
###############################################
c.min_width_event_onphase = c.sample_rate//2 # set minimum withd of an onphase event; default: 0.5 seconds
c.min_width_event_upphase = c.sample_rate//4 # set minimum width of upphase event; default: 0.25 seconds

############# PARAMTERS TO TWEAK ##############
#     1. Cutoff for calling somthing a spike:
#        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
#                                        higher -> less events; lower -> more events
#                                        start at default and increase if data is very noisy and getting too many noise-events
c.min_thresh_std_onphase = 2.5      # set the minimum thrshold for onphase detection; defatul 2.5
c.min_thresh_std_upphase = 2.5      # set the minimum thershold for uppohase detection; default: 2.5

#     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
#        This is stored in high_cutoff: default 0.5 to 1.0
#        The lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces (but we loose some temporal precision)
c.high_cutoff = 0.5              

#     3. Removing bleaching and drift artifacts using polynomial fits
#        This is stored in detrend_model_order
c.detrend_model_order = 1 # 1-5 polynomial fit


################################################
########### RUN BINARIZATION STEP ##############
################################################
# 
c.binarize_fluorescence()

print (" ... DONE ...")

  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (522, 55741)
         self.Fneu (neuropile):  (522, 55741)
         self.iscell (Suite2p cell classifier output):  (568, 2)
              of which number of good cells:  (522,)
         self.spks (deconnvoved spikes):  (522, 55741)
         self.stat (footprints structure):  (522,)
         mean std over all cells :  29.5

  Binarization parameters: 
        low pass filter low cuttoff:  0.5 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_onphase:  2.5
        min_thresh_std_upphase:  2.5
        min_width_event_onphase:  15
        min_width_event_upphase:  7
        min_width_event_oasis:  2
        min_event_amplitude:  1


model filter: remove bleaching or trends: 100%|██████████| 522/522 [00:00<00:00, 3283.68it/s]
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 522/522 [00:00<00:00, 1112.44it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 522/522 [00:01<00:00, 516.47it/s]


   Oasis based binarization skipped by default ... 
 ... DONE ...


In [11]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
#fname = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/nathalie/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/renan/renan/binarized_traces.npz'
# 
fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')

c = calcium.Calcium()
c.fname = fname
c.data_dir = data_dir
c.recompute_binarization = False
c.load_binarization()

# pick a random cell to visuzlie
cell_id = 0
scale = 100

# 
c.plot_cell_binarization(cell_id, scale)


In [10]:
######################################################
############# SHOW RASTERS ###########################
######################################################
fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'

c = calcium.Calcium()
c.data_dir = data_dir
c.fname = fname
c.recompute_binarization = False
c.load_binarization()

c.show_rasters()
